<a href="https://colab.research.google.com/github/fhazraty/LearnHuggingFace/blob/main/FoodNotFood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers datasets huggingface_hub accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 12.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [2]:
from huggingface_hub import login
login()

In [3]:
# Install dependencies (this is mostly for Google Colab, as the other dependences are available by default in Colab)
try:
  import datasets, evaluate, accelerate
  import gradio as gr
except ModuleNotFoundError:
  !pip install -U datasets evaluate accelerate gradio # -U stands for "upgrade" so we'll get the latest version by default
  import datasets, evaluate, accelerate
  import gradio as gr

import random

import numpy as np
import pandas as pd

import torch
import transformers

print(f"Using transformers version: {transformers.__version__}")
print(f"Using datasets version: {datasets.__version__}")
print(f"Using torch version: {torch.__version__}")



Using transformers version: 4.57.3
Using datasets version: 4.4.2
Using torch version: 2.9.0+cu126


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/11.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/250 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 250
    })
})

In [24]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import numpy as np

# 1) Load
dataset = load_dataset("mrdbourke/learn_hf_food_not_food_image_captions")

# 2) Label mapping (فقط اگر label رشته‌ای باشد)
label2id = {"not_food": 0, "food": 1}
id2label = {0: "not_food", 1: "food"}

def fix_label(example):
    # اگر label رشته‌ای بود تبدیلش کن؛ اگر عددی بود دست نزن
    if isinstance(example["label"], str):
        example["label"] = label2id[example["label"]]
    # اگر به شکل لیست آمد (خیلی نادر) یک عنصرش را بردار
    if isinstance(example["label"], list):
        example["label"] = example["label"][0]
        if isinstance(example["label"], str):
            example["label"] = label2id[example["label"]]
    return example

dataset = dataset.map(fix_label)

# 3) Tokenize
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

tokenized = dataset.map(tokenize_function, batched=True)

# 4) فقط ستون‌های لازم را نگه دار (این خط کلید حل مشکل است)
keep_cols = ["input_ids", "attention_mask", "label"]
tokenized = tokenized["train"].remove_columns([c for c in tokenized["train"].column_names if c not in keep_cols])

# 5) train/test split
split = tokenized.train_test_split(test_size=0.2, seed=42)
train_dataset = split["train"]
eval_dataset  = split["test"]

# 6) فرمت torch
train_dataset = train_dataset.with_format("torch")
eval_dataset  = eval_dataset.with_format("torch")

# 7) Model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

# 8) TrainingArguments (نسخه جدید)
training_args = TrainingArguments(
    output_dir="./food_not_food_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none"
)

# 9) Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# 10) TRAIN
trainer.train()


Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-4139801799.py:75: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,0.313224
2,No log,0.140497
3,No log,0.097678


TrainOutput(global_step=39, training_loss=0.2950545824491061, metrics={'train_runtime': 25.2082, 'train_samples_per_second': 23.802, 'train_steps_per_second': 1.547, 'total_flos': 19870109798400.0, 'train_loss': 0.2950545824491061, 'epoch': 3.0})

In [26]:
from transformers import pipeline

classifier = pipeline("text-classification", model=trainer.model, tokenizer=tokenizer)

classifier("A plate of spaghetti with cheese")


Device set to use cuda:0


[{'label': 'food', 'score': 0.8309544324874878}]